# <span style="color:blue"> UNet testing saving .png files

**Content under Creative Commons Attribution license CC-BY-NC-SA 4.0**  
**Code under GNU-GPL v3 License**  
**© 2023 Francesco Chiumento**

- Through this notebook, it is possible to utilize a previously saved checkpoint of the UNet neural network to segment femoral cartilages in 2D images extracted from MRI scans of a set of patients. In this version of the code, it is possible to provide the ground truth of the respective binary masks of the patients to visualize the qualitative results of the DICE indices of the obtained 2D segmentations;

- The ultimate goal is to obtain reconstructed segmentations in three-dimensional volumes from the 2D segmentations.

In [ ]:
%load_ext autoreload
%autoreload 2

## Library Import and Module Definition

In [ ]:
from slice_extraction import process_directory, clean_folder
from unet_segmentation import run_segmentation
from compositionMha import combine_slices_to_mha
from ipywidgets import Checkbox, VBox

import torch
import os

## Toggle DICE Calculation

- When value is set to True, the option to utilize the binary ground truth masks for qualitative evaluation (using the Dice index) of the neural network's performance is available;

- In the absence of ground truth masks, set value to False.

In [ ]:
use_masks_checkbox = Checkbox(
    value=True,
    description='Use Masks for DICE Calculation',
    disabled=False
)

display(VBox([use_masks_checkbox]))

## Path and Settings Definition

- In *images_dir* and *masks_dir* (if masks are available), place the images and their corresponding binary masks from which to extract individual slices for segmentation;
- In *images_slices* and *masks_slices*, individual slices extracted from the images and their corresponding masks (if available) will be saved;
- checkpoint_path represents the saved checkpoint obtained from training and validating the neural network;
- *predictions* is the folder where the 2D segmentations of individual slices will be saved;
- *segmentations* is the folder where the reconstructed 3D segmentations obtained from the *predictions* folder will be saved.

In [ ]:
# Define paths and settings

images_dir = r"your/path/here"
masks_dir = r"your/path/here"
images_slices = r"your/path/here"
masks_slices = r"your/path/here"
checkpoint_path = r"your/path/here"
predictions = r"your/path/here"
segmentations = r"your/path/here"

spacing = (0.4121, 0.4121, 0.4)  # Update according to your image spacing

should_flip = True # Option to flip the images if needed

## Patient Directory Listing

In [ ]:
patient_dirs = os.listdir(images_dir)

## Device Selection

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Patient Data Processing Loop

In [ ]:
for patient in patient_dirs:
    image_mha_path = os.path.join(images_dir, patient)
    mask_mha_path = os.path.join(masks_dir, patient)
    
    # Clean the folders for slices and predictions before each patient
    clean_folder(images_slices)
    clean_folder(masks_slices)
    clean_folder(predictions)

    # Call the updated function with the specific file paths
    process_directory(image_mha_path, mask_mha_path, images_slices, masks_slices, use_masks=use_masks_checkbox.value)

    run_segmentation(checkpoint_path, images_slices, masks_slices, predictions, device, use_masks=use_masks_checkbox.value)

    combine_slices_to_mha(predictions, os.path.join(segmentations, f"{patient}_segmentation.mha"), spacing, should_flip)
   
    print(f"The process has been completed for the patient {patient}.")


## Computer system details

In [ ]:
%reload_ext watermark
%watermark